# Flight Search Agent Tutorial

This notebook demonstrates the Agent Catalog flight search agent with fixed parameter mapping and robust ReAct artifact cleaning.


## Setup and Imports

Import all necessary modules with parameter mapping.


In [1]:
import base64
import getpass
import inspect
import json
import logging
import os
import re
import sys
import time
from datetime import timedelta
from typing import Any

import agentc
import agentc_langgraph.agent
import agentc_langgraph.graph
import dotenv
import langchain_core.messages
import langchain_core.runnables
import langchain_openai.chat_models
import langgraph.graph
import openai
import requests
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Setup logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Suppress verbose logging
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("agentc_core").setLevel(logging.WARNING)

# Load environment variables
dotenv.load_dotenv(override=True)


All imports completed!


## Environment Setup

Setup environment variables and initialization functions.


In [2]:
def setup_capella_ai_config():
    """Setup Capella AI configuration - requires environment variables to be set."""
    # Verify required environment variables are set (no defaults)
    required_capella_vars = [
        "CB_USERNAME",
        "CB_PASSWORD",
        "CAPELLA_API_ENDPOINT",
        "CAPELLA_API_EMBEDDING_MODEL",
        "CAPELLA_API_LLM_MODEL",
    ]
    missing_vars = [var for var in required_capella_vars if not os.getenv(var)]
    if missing_vars:
        msg = f"Missing required Capella AI environment variables: {missing_vars}"
        raise ValueError(msg)

    return {
        "endpoint": os.getenv("CAPELLA_API_ENDPOINT"),
        "embedding_model": os.getenv("CAPELLA_API_EMBEDDING_MODEL"),
        "llm_model": os.getenv("CAPELLA_API_LLM_MODEL"),
        "dimensions": 4096,
    }


def test_capella_connectivity():
    """Test connectivity to Capella AI services."""
    try:
        endpoint = os.getenv("CAPELLA_API_ENDPOINT")
        if not endpoint:
            logger.warning("CAPELLA_API_ENDPOINT not configured")
            return False

        # Test basic HTTP connectivity
        logger.info("Testing Capella AI connectivity...")
        response = requests.get(f"{endpoint}/health", timeout=10)
        if response.status_code != 200:
            logger.warning(f"Capella AI health check failed: {response.status_code}")

        # Test embedding model (requires API key)
        if os.getenv("CB_USERNAME") and os.getenv("CB_PASSWORD"):
            api_key = base64.b64encode(
                f"{os.getenv('CB_USERNAME')}:{os.getenv('CB_PASSWORD')}".encode()
            ).decode()

            headers = {"Authorization": f"Basic {api_key}", "Content-Type": "application/json"}

            # Test embedding
            embedding_data = {
                "model": os.getenv(
                    "CAPELLA_API_EMBEDDING_MODEL", "intfloat/e5-mistral-7b-instruct"
                ),
                "input": "test connectivity",
            }

            embedding_response = requests.post(
                f"{endpoint}/v1/embeddings", headers=headers, json=embedding_data, timeout=30
            )

            if embedding_response.status_code == 200:
                embed_result = embedding_response.json()
                embed_dims = len(embed_result["data"][0]["embedding"])
                logger.info(f"✅ Capella AI embedding test successful - dimensions: {embed_dims}")

                if embed_dims != 4096:
                    logger.warning(f"Expected 4096 dimensions, got {embed_dims}")
                    return False
            else:
                logger.warning(
                    f"Capella AI embedding test failed: {embedding_response.status_code}"
                )
                return False

            # Test LLM
            llm_data = {
                "model": os.getenv("CAPELLA_API_LLM_MODEL", "meta-llama/Llama-3.1-8B-Instruct"),
                "messages": [{"role": "user", "content": "Hello"}],
                "max_tokens": 10,
            }

            llm_response = requests.post(
                f"{endpoint}/v1/chat/completions", headers=headers, json=llm_data, timeout=30
            )

            if llm_response.status_code == 200:
                logger.info("✅ Capella AI LLM test successful")
            else:
                logger.warning(f"Capella AI LLM test failed: {llm_response.status_code}")
                return False

        logger.info("✅ Capella AI connectivity tests completed successfully")
        return True

    except Exception as e:
        logger.warning(f"Capella AI connectivity test failed: {e}")
        return False


def _set_if_undefined(var: str):
    if os.environ.get(var) is None:
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")


def setup_environment():
    """Setup required environment variables with defaults."""
    # Setup Capella AI configuration first
    setup_capella_ai_config()

    # Required variables
    required_vars = ["OPENAI_API_KEY", "CB_CONN_STRING", "CB_USERNAME", "CB_PASSWORD", "CB_BUCKET"]
    for var in required_vars:
        _set_if_undefined(var)

    defaults = {
        "CB_CONN_STRING": "couchbase://localhost",
        "CB_USERNAME": "Administrator",
        "CB_PASSWORD": "password",
        "CB_BUCKET": "travel-sample",
    }

    for key, default_value in defaults.items():
        if not os.environ.get(key):
            os.environ[key] = input(f"Enter {key} (default: {default_value}): ") or default_value

    os.environ["CB_INDEX"] = os.getenv("CB_INDEX", "flight_policies_index")
    os.environ["CB_SCOPE"] = os.getenv("CB_SCOPE", "agentc_data")
    os.environ["CB_COLLECTION"] = os.getenv("CB_COLLECTION", "flight_policies")

    # Test Capella AI connectivity
    test_capella_connectivity()


setup_environment()

2025-07-13 12:42:18,497 - __main__ - INFO - Testing Capella AI connectivity...
2025-07-13 12:42:21,145 - __main__ - INFO - ✅ Capella AI embedding test successful - dimensions: 4096
2025-07-13 12:42:22,529 - __main__ - INFO - ✅ Capella AI LLM test successful
2025-07-13 12:42:22,544 - __main__ - INFO - ✅ Capella AI connectivity tests completed successfully


Environment configured


## CouchbaseClient Class

Define the CouchbaseClient for all database operations.


In [3]:
class CouchbaseClient:
    """Centralized Couchbase client for all database operations."""

    def __init__(self, conn_string: str, username: str, password: str, bucket_name: str):
        """Initialize Couchbase client with connection details."""
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        """Establish connection to Couchbase cluster."""
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)
            # Use WAN profile for better timeout handling with remote clusters
            options.apply_profile("wan_development")
            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=10))
            logger.info("Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            msg = f"Failed to connect to Couchbase: {e!s}"
            raise ConnectionError(msg)

    def setup_collection(self, scope_name: str, collection_name: str):
        """Setup bucket, scope and collection all in one function."""
        try:
            if not self.cluster:
                self.connect()

            if not self.bucket:
                try:
                    self.bucket = self.cluster.bucket(self.bucket_name)
                    logger.info(f"Bucket '{self.bucket_name}' exists")
                except Exception:
                    logger.info(f"Creating bucket '{self.bucket_name}'...")
                    bucket_settings = CreateBucketSettings(
                        name=self.bucket_name,
                        bucket_type="couchbase",
                        ram_quota_mb=1024,
                        flush_enabled=True,
                        num_replicas=0,
                    )
                    self.cluster.buckets().create_bucket(bucket_settings)
                    time.sleep(5)
                    self.bucket = self.cluster.bucket(self.bucket_name)
                    logger.info(f"Bucket '{self.bucket_name}' created successfully")

            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                logger.info(f"Creating scope '{scope_name}'...")
                bucket_manager.create_scope(scope_name)
                logger.info(f"Scope '{scope_name}' created successfully")

            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name
                and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if not collection_exists:
                logger.info(f"Creating collection '{collection_name}'...")
                bucket_manager.create_collection(scope_name, collection_name)
                logger.info(f"Collection '{collection_name}' created successfully")

            collection = self.bucket.scope(scope_name).collection(collection_name)
            time.sleep(3)

            try:
                self.cluster.query(
                    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                ).execute()
                logger.info("Primary index created successfully")
            except Exception as e:
                logger.warning(f"Error creating primary index: {e!s}")

            collection_key = f"{scope_name}.{collection_name}"
            self._collections[collection_key] = collection

            logger.info(f"Collection setup complete for {scope_name}.{collection_name}")
            return collection

        except Exception as e:
            msg = f"Error setting up collection: {e!s}"
            raise RuntimeError(msg)

    def get_collection(self, scope_name: str, collection_name: str):
        """Get a collection, creating it if it doesn't exist."""
        collection_key = f"{scope_name}.{collection_name}"
        if collection_key not in self._collections:
            self.setup_collection(scope_name, collection_name)
        return self._collections[collection_key]

    def clear_scope(self, scope_name: str):
        """Clear all collections in the specified scope."""
        try:
            if not self.bucket:
                if not self.cluster:
                    self.connect()
                self.bucket = self.cluster.bucket(self.bucket_name)

            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()

            target_scope = None
            for scope in scopes:
                if scope.name == scope_name:
                    target_scope = scope
                    break

            if not target_scope:
                logger.info(f"Scope '{scope_name}' does not exist, nothing to clear")
                return

            for collection in target_scope.collections:
                try:
                    delete_query = (
                        f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection.name}`"
                    )
                    self.cluster.query(delete_query).execute()
                    logger.info(f"Cleared collection '{collection.name}' in scope '{scope_name}'")
                except Exception as e:
                    logger.warning(f"Could not clear collection '{collection.name}': {e}")

            logger.info(f"Cleared all collections in scope '{scope_name}'")

        except Exception as e:
            logger.warning(f"Could not clear scope '{scope_name}': {e}")

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        """Setup vector search index for the specified scope."""
        try:
            if not self.bucket:
                msg = "Bucket not initialized. Call setup_collection first."
                raise RuntimeError(msg)

            scope_index_manager = self.bucket.scope(scope_name).search_indexes()
            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                logger.info(f"Creating vector search index '{index_name}'...")
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"Vector search index '{index_name}' created successfully")
            else:
                logger.info(f"Vector search index '{index_name}' already exists")
        except Exception as e:
            msg = f"Error setting up vector search index: {e!s}"
            raise RuntimeError(msg)

    def load_flight_data(self):
        """Load flight data from the enhanced flight_data.py file."""
        try:
            # Import flight data
            sys.path.append(os.path.join(os.path.dirname(__file__), "data"))
            from flight_data import get_all_flight_data

            flight_data = get_all_flight_data()

            # Convert to text format for vector store
            flight_texts = []
            for item in flight_data:
                text = f"{item['title']} - {item['content']}"
                flight_texts.append(text)

            return flight_texts
        except Exception as e:
            msg = f"Error loading flight data: {e!s}"
            raise ValueError(msg)

    def setup_vector_store(
        self, scope_name: str, collection_name: str, index_name: str, embeddings
    ):
        """Setup vector store with flight data."""
        try:
            if not self.cluster:
                msg = "Cluster not connected. Call connect first."
                raise RuntimeError(msg)

            vector_store = CouchbaseVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embedding=embeddings,
                index_name=index_name,
            )

            # Load flight data - single attempt
            try:
                flight_data = self.load_flight_data()
                vector_store.add_texts(texts=flight_data, batch_size=10)
                logger.info("Flight data loaded into vector store successfully")
            except Exception as e:
                logger.exception(f"Failed to load flight data: {e}")
                logger.warning("Vector store created but data not loaded.")

            return vector_store
        except Exception as e:
            msg = f"Error setting up vector store: {e!s}"
            raise ValueError(msg)



CouchbaseClient defined


## Parameter Mapper

Define the ParameterMapper class with ReAct artifact cleaning.


In [4]:
class ParameterMapper:
    """Intelligent parameter mapper using LLM with guardrail-safe prompts."""

    def __init__(self, chat_model: ChatOpenAI):
        self.chat_model = chat_model

        # Common parameter synonyms for flight tools
        self.parameter_synonyms = {
            "source_airport": ["departure_airport", "origin", "from", "origin_airport", "start"],
            "destination_airport": ["arrival_airport", "destination", "to", "dest", "end"],
            "departure_date": ["date", "travel_date", "dep_date", "when"],
            "return_date": ["return", "return_date", "back_date"],
            "passengers": ["pax", "travelers", "people", "passenger_count"],
            "flight_class": ["class", "cabin", "service_class", "ticket_class"],
        }

    def get_function_parameters(self, func) -> set[str]:
        """Extract parameter names from function signature."""
        try:
            sig = inspect.signature(func)
            return set(sig.parameters.keys())
        except Exception:
            logger.exception("Error getting function parameters")
            return set()

    def map_parameters_smart(
        self, tool_name: str, raw_args: dict[str, Any], func
    ) -> dict[str, Any]:
        """
        Smart parameter mapping using LLM to understand parameter intent.

        Args:
            tool_name: Name of the tool being called
            raw_args: Raw arguments from LLM
            func: Function object to get expected parameters

        Returns:
            Mapped parameters ready for function call
        """
        try:
            # Get expected parameters from function signature
            expected_params = self.get_function_parameters(func)

            # If parameters already match, return as-is
            if set(raw_args.keys()).issubset(expected_params):
                return raw_args

            # Use LLM to map parameters intelligently with guardrail-safe prompts
            mapped_params = self._llm_parameter_mapping(tool_name, raw_args, expected_params)

            # Add tool-specific defaults
            mapped_params = self._add_tool_defaults(tool_name, mapped_params)

            # Filter to only valid parameters
            return {k: v for k, v in mapped_params.items() if k in expected_params}


        except Exception:
            logger.exception("Error in smart parameter mapping")
            # Fallback to synonym-based mapping
            return self._fallback_synonym_mapping(raw_args, expected_params)

    def _llm_parameter_mapping(
        self, tool_name: str, raw_args: dict[str, Any], expected_params: set[str]
    ) -> dict[str, Any]:
        """Use LLM to map parameters with minimal, guardrail-safe prompts."""

        # Ultra-minimal prompt to avoid guardrail violations
        system_prompt = f"""Map parameters to: {list(expected_params)}
Output only valid JSON."""

        user_prompt = f"""Input: {json.dumps(raw_args)}
Tool: {tool_name}"""

        try:
            # Primary LLM call with minimal prompt
            mapped_params = self._safe_llm_call(system_prompt, user_prompt)
            if mapped_params:
                return mapped_params

        except Exception as e:
            logger.warning(f"LLM parameter mapping failed: {e}")

        # Fallback to synonym-based mapping
        return self._fallback_synonym_mapping(raw_args, expected_params)

    def _safe_llm_call(self, system_prompt: str, user_prompt: str) -> dict | None:
        """Make safe LLM call with robust JSON parsing."""
        try:
            messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]

            response = self.chat_model.invoke(messages)
            content = response.content.strip()

            # Try multiple JSON parsing methods
            return self._parse_json_response(content)

        except Exception as e:
            logger.warning(f"Safe LLM call failed: {e}")
            return None

    def _parse_json_response(self, content: str) -> dict | None:
        """Parse JSON response with multiple fallback methods."""

        logger.debug(f"Parsing JSON response: '{content[:200]}...' (length: {len(content)})")

        # Method 1: Direct JSON parsing
        try:
            return json.loads(content)
        except json.JSONDecodeError as e:
            logger.debug(f"Direct JSON parsing failed: {e}")

        # Method 2: Extract from code blocks
        try:
            if "```json" in content:
                json_content = content.split("```json")[1].split("```")[0].strip()
                return json.loads(json_content)
            if "```" in content:
                json_content = content.split("```")[1].strip()
                return json.loads(json_content)
        except (json.JSONDecodeError, IndexError):
            pass

        # Method 3: Find JSON-like patterns
        try:
            # Look for balanced {.*} patterns
            json_pattern = r"\{[^{}]*\}"
            matches = re.findall(json_pattern, content)
            if matches:
                return json.loads(matches[0])
        except (json.JSONDecodeError, IndexError):
            pass

        # Method 3b: More complex JSON pattern with nested braces
        try:
            # Find JSON with proper brace matching
            start = content.find("{")
            if start != -1:
                brace_count = 0
                for i, char in enumerate(content[start:], start):
                    if char == "{":
                        brace_count += 1
                    elif char == "}":
                        brace_count -= 1
                        if brace_count == 0:
                            json_content = content[start : i + 1]
                            return json.loads(json_content)
        except (json.JSONDecodeError, ValueError):
            pass

        # Method 4: Handle truncated JSON (common at character 168)
        try:
            # If content appears truncated, try to find the last complete JSON object
            if len(content) >= 160:  # Near the problematic character range
                # Look for the last complete brace pair
                last_close_brace = content.rfind("}")
                if last_close_brace > 0:
                    # Find the matching opening brace
                    brace_count = 0
                    for i in range(last_close_brace, -1, -1):
                        if content[i] == "}":
                            brace_count += 1
                        elif content[i] == "{":
                            brace_count -= 1
                            if brace_count == 0:
                                truncated_json = content[i : last_close_brace + 1]
                                logger.debug(f"Trying truncated JSON: '{truncated_json}'")
                                return json.loads(truncated_json)
        except (json.JSONDecodeError, ValueError):
            pass

        # Method 5: Clean and retry
        try:
            # Remove extra whitespace and try again
            cleaned = re.sub(r"\s+", " ", content.strip())
            return json.loads(cleaned)
        except json.JSONDecodeError:
            pass

        # Method 6: Emergency fallback - try to construct basic JSON from visible patterns
        try:
            # Look for key-value pairs and construct basic JSON
            if "source_airport" in content and "destination_airport" in content:
                # Try to extract airport codes with regex
                source_match = re.search(r'"source_airport"\s*:\s*"([^"]*)"', content)
                dest_match = re.search(r'"destination_airport"\s*:\s*"([^"]*)"', content)

                if source_match and dest_match:
                    fallback_json = {
                        "source_airport": source_match.group(1),
                        "destination_airport": dest_match.group(1),
                    }
                    logger.debug(f"Emergency fallback JSON: {fallback_json}")
                    return fallback_json
        except Exception:
            pass

        logger.warning(
            f"Failed to parse JSON response (length: {len(content)}): {content[:200]}..."
        )
        return None

    def _fallback_synonym_mapping(
        self, raw_args: dict[str, Any], expected_params: set[str]
    ) -> dict[str, Any]:
        """Fallback to synonym-based mapping if LLM fails."""
        mapped = {}

        for expected_param in expected_params:
            # Check if parameter exists directly
            if expected_param in raw_args:
                mapped[expected_param] = raw_args[expected_param]
                continue

            # Check synonyms
            synonyms = self.parameter_synonyms.get(expected_param, [])
            for synonym in synonyms:
                if synonym in raw_args:
                    mapped[expected_param] = raw_args[synonym]
                    break

        return mapped

    def _add_tool_defaults(self, tool_name: str, params: dict[str, Any]) -> dict[str, Any]:
        """Add tool-specific defaults for single user system."""

        if tool_name == "save_flight_booking":
            # Default departure date if missing
            if "departure_date" not in params:
                params["departure_date"] = "tomorrow"

            # Default passengers if missing
            if "passengers" not in params:
                params["passengers"] = 1

            # Default flight class if missing
            if "flight_class" not in params:
                params["flight_class"] = "economy"

        elif tool_name == "retrieve_flight_bookings":
            # No customer_id needed for single user system
            pass

        return params

    def extract_airports_from_text(self, text: str) -> dict[str, str | None]:
        """Extract airport codes from text using guardrail-safe LLM calls."""

        # Ultra-minimal prompt for location code extraction
        system_prompt = (
            "Extract location codes. Return JSON with source_airport and destination_airport."
        )
        user_prompt = f"Text: {text}"

        try:
            # Try LLM extraction with minimal prompt
            result = self._safe_llm_call(system_prompt, user_prompt)
            if result and isinstance(result, dict):
                # Validate and clean the result
                cleaned_result = {}
                for key in ["source_airport", "destination_airport"]:
                    value = result.get(key)
                    if value and isinstance(value, str) and len(value) == 3:
                        cleaned_result[key] = value.upper()
                    else:
                        cleaned_result[key] = None
                return cleaned_result

        except Exception as e:
            logger.warning(f"LLM airport extraction failed: {e}")

        # Fallback to pattern matching
        return self._fallback_airport_extraction(text)

    def _fallback_airport_extraction(self, text: str) -> dict[str, str | None]:
        """Fallback airport extraction using regex patterns."""
        result = {"source_airport": None, "destination_airport": None}

        # Find 3-letter codes
        airport_codes = re.findall(r"\b[A-Z]{3}\b", text.upper())

        if len(airport_codes) >= 2:
            result["source_airport"] = airport_codes[0]
            result["destination_airport"] = airport_codes[1]
        elif len(airport_codes) == 1:
            # Try to determine if it's source or destination
            if any(word in text.lower() for word in ["from", "origin"]):
                result["source_airport"] = airport_codes[0]
            elif any(word in text.lower() for word in ["to", "destination"]):
                result["destination_airport"] = airport_codes[0]

        return result

    def map_positional_args(self, tool_name: str, args: tuple, func) -> dict[str, Any]:
        """Map positional arguments from ReAct agent to function parameters."""
        try:
            expected_params = self.get_function_parameters(func)

            # Map positional args to expected parameter names
            mapped = {}

            if tool_name == "lookup_flight_info":
                # Expects source_airport, destination_airport
                if len(args) >= 2:
                    mapped["source_airport"] = args[0]
                    mapped["destination_airport"] = args[1]
                elif len(args) == 1:
                    # Try to extract both from single string
                    airports = self.extract_airports_from_text(args[0])
                    mapped.update(airports)

            elif tool_name == "save_flight_booking":
                # Map based on position and add defaults
                if len(args) >= 2:
                    mapped["source_airport"] = args[0]
                    mapped["destination_airport"] = args[1]
                if len(args) >= 3:
                    mapped["departure_date"] = args[2]

                # Add defaults for missing parameters
                mapped = self._add_tool_defaults(tool_name, mapped)

            elif tool_name == "search_flight_policies":
                # Single query parameter
                if len(args) >= 1:
                    mapped["query"] = " ".join(args)

            # Filter to only valid parameters
            return {k: v for k, v in mapped.items() if k in expected_params}

        except Exception:
            logger.exception("Error mapping positional args for %s", tool_name)
            return {}

    def map_string_input(self, tool_name: str, input_str: str, func) -> dict[str, Any]:
        """Map single string input to function parameters."""
        try:
            expected_params = self.get_function_parameters(func)
            mapped = {}

            logger.debug(
                f"Parameter mapping for {tool_name}: input='{input_str}', expected={expected_params}"
            )

            # Enhanced cleaning for ReAct parsing artifacts
            clean_input = self._clean_react_artifacts(input_str)

            # Debug logging for edge cases
            if input_str != clean_input:
                logger.debug(
                    f"Cleaned ReAct artifacts for {tool_name}: '{input_str}' -> '{clean_input}'"
                )

            if tool_name == "lookup_flight_info":
                # Handle comma-separated format: "JFK,LAX,tomorrow"
                parts = [part.strip() for part in clean_input.split(",")]

                if len(parts) >= 2:
                    # Direct airport codes
                    mapped["source_airport"] = parts[0].upper()
                    mapped["destination_airport"] = parts[1].upper()
                else:
                    # Try to extract airports from string
                    airports = self.extract_airports_from_text(clean_input)
                    mapped.update(airports)

            elif tool_name == "save_flight_booking":
                # Handle comma-separated format: "SOURCE,DEST,DATE,PASSENGERS,CLASS"
                parts = [part.strip() for part in clean_input.split(",")]

                if len(parts) >= 2:
                    mapped["source_airport"] = parts[0].upper()
                    mapped["destination_airport"] = parts[1].upper()

                    # Handle positional parameters
                    if len(parts) >= 3:
                        mapped["departure_date"] = parts[2]
                    if len(parts) >= 4:
                        # Try to parse passengers as integer
                        try:
                            mapped["passengers"] = int(parts[3])
                        except ValueError:
                            # If not integer, check if it contains a number
                            numbers = re.findall(r"\d+", parts[3])
                            if numbers:
                                mapped["passengers"] = int(numbers[0])
                    if len(parts) >= 5:
                        # Enhanced cleaning for flight class with ReAct artifacts
                        flight_class = self._clean_flight_class(parts[4])
                        if flight_class:
                            mapped["flight_class"] = flight_class
                else:
                    # Try to extract flight info from text
                    airports = self.extract_airports_from_text(clean_input)
                    mapped.update(airports)

                # Add defaults
                mapped = self._add_tool_defaults(tool_name, mapped)

            elif tool_name == "search_flight_policies":
                # Use cleaned string as query
                mapped["query"] = clean_input

            elif tool_name == "retrieve_flight_bookings":
                # No parameters needed for single user system
                pass

            # Filter to only valid parameters
            final_mapped = {
                k: v for k, v in mapped.items() if k in expected_params and v is not None
            }

            logger.debug(f"Parameter mapping result for {tool_name}: {final_mapped}")

            if not final_mapped:
                logger.warning(
                    f"No valid parameters mapped for {tool_name} with input '{input_str}'"
                )

            return final_mapped

        except Exception:
            logger.exception("Error mapping string input for %s", tool_name)
            return {}

    def _clean_react_artifacts(self, input_str: str) -> str:
        """Clean ReAct parsing artifacts from input string."""
        if not input_str:
            return ""

        # Remove common ReAct artifacts
        clean_str = input_str

        # Enhanced cleaning for ReAct artifacts - handle multi-line patterns
        # Remove trailing quotes and observation artifacts (case insensitive)
        clean_str = re.sub(
            r'["\']?\s*\n?\s*observation.*$', "", clean_str, flags=re.IGNORECASE | re.DOTALL
        )

        # Remove newlines followed by any text (common ReAct artifact)
        clean_str = re.sub(r"\n.*$", "", clean_str, flags=re.DOTALL)

        # Remove leading/trailing quotes and whitespace
        clean_str = clean_str.strip().strip("\"'").strip()

        # Handle specific ReAct patterns like \"None\nObservation\"
        if clean_str.lower().startswith("none"):
            # Extract just \"none\" if it starts with none followed by artifacts
            clean_str = "none"

        return clean_str

    def _clean_flight_class(self, flight_class_str: str) -> str:
        """Clean flight class parameter with enhanced artifact removal."""
        if not flight_class_str:
            return ""

        # Start with basic cleaning
        cleaned = flight_class_str.strip().lower()

        # Remove quotes
        cleaned = cleaned.strip("\"'")

        # Remove observation artifacts (case insensitive)
        cleaned = re.sub(r'\s*["\']?\s*observation.*$', "", cleaned, flags=re.IGNORECASE)

        # Remove newlines and everything after
        cleaned = re.sub(r"\n.*$", "", cleaned)

        # Remove any remaining special characters at the end
        cleaned = re.sub(r"[^a-zA-Z]+$", "", cleaned)

        # Final trim
        cleaned = cleaned.strip()

        # Validate against known flight classes
        valid_classes = ["economy", "business", "first", "premium"]
        if cleaned in valid_classes:
            return cleaned

        # If not exact match, try to find closest match
        for valid_class in valid_classes:
            if valid_class in cleaned:
                return valid_class

        return cleaned  # Return as-is if no match found


## Test Parameter Mapping

Test the parameter mapping functionality.


In [5]:
# Quick test of parameter mapping
from langchain_openai import ChatOpenAI

# Test the parameter mapper
chat_model = ChatOpenAI(model="gpt-4o", temperature=0)
parameter_mapper = ParameterMapper(chat_model)


# Mock function for testing
def mock_save_booking(
    source_airport: str,
    destination_airport: str,
    departure_date: str,
    passengers: int = 1,
    flight_class: str = "economy",
):
    return f"Booked {passengers} passengers from {source_airport} to {destination_airport} on {departure_date} in {flight_class}"


# Test cases
test_inputs = [
    "JFK,LAX,tomorrow,1,economy",
    'JFK,LAX,tomorrow,1,economy"\nobservation',  # ReAct artifact case
    "LAX,JFK,next week,2,business",
]

for test_input in test_inputs:
    result = parameter_mapper.map_string_input("save_flight_booking", test_input, mock_save_booking)
    if result:
        booking_result = mock_save_booking(**result)



Testing: 'JFK,LAX,tomorrow,1,economy'
Mapped parameters: {'source_airport': 'JFK', 'destination_airport': 'LAX', 'departure_date': 'tomorrow', 'passengers': 1, 'flight_class': 'economy'}
Result: Booked 1 passengers from JFK to LAX on tomorrow in economy

Testing: 'JFK,LAX,tomorrow,1,economy"
observation'
Mapped parameters: {'source_airport': 'JFK', 'destination_airport': 'LAX', 'departure_date': 'tomorrow', 'passengers': 1, 'flight_class': 'economy'}
Result: Booked 1 passengers from JFK to LAX on tomorrow in economy

Testing: 'LAX,JFK,next week,2,business'
Mapped parameters: {'source_airport': 'LAX', 'destination_airport': 'JFK', 'departure_date': 'next week', 'passengers': 2, 'flight_class': 'business'}
Result: Booked 2 passengers from LAX to JFK on next week in business

Parameter mapping tests complete


## Agent Classes

Define the FlightSearchAgent with parameter mapping integration.


In [6]:
class FlightSearchState(agentc_langgraph.agent.State):
    """State for flight search conversations - single user system."""

    query: str
    resolved: bool
    search_results: list[dict]


class FlightSearchAgent(agentc_langgraph.agent.ReActAgent):
    """Flight search agent with robust parameter mapping."""

    def __init__(self, catalog: agentc.Catalog, span: agentc.Span):
        """Initialize the flight search agent."""

        # Try Capella AI first, fallback to OpenAI
        chat_model = None
        try:
            if (
                os.getenv("CB_USERNAME")
                and os.getenv("CB_PASSWORD")
                and os.getenv("CAPELLA_API_ENDPOINT")
                and os.getenv("CAPELLA_API_LLM_MODEL")
            ):
                # Create API key for Capella AI
                api_key = base64.b64encode(
                    f"{os.getenv('CB_USERNAME')}:{os.getenv('CB_PASSWORD')}".encode()
                ).decode()

                chat_model = ChatOpenAI(
                    model=os.getenv("CAPELLA_API_LLM_MODEL"),
                    api_key=api_key,
                    base_url=f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                    temperature=0.0,
                    max_tokens=512,
                    timeout=30,
                )
                logger.info("✅ Using Capella AI for LLM")
            else:
                msg = "Capella AI credentials not available"
                raise ValueError(msg)

        except Exception as e:
            logger.warning(f"Capella AI LLM failed, falling back to OpenAI: {e}")
            model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
            chat_model = langchain_openai.chat_models.ChatOpenAI(
                model=model_name, temperature=0.0, max_tokens=512, timeout=30
            )
            logger.info("✅ Using OpenAI for LLM (fallback)")

        super().__init__(
            chat_model=chat_model, catalog=catalog, span=span, prompt_name="flight_search_assistant"
        )

    def _invoke(
        self,
        span: agentc.Span,
        state: FlightSearchState,
        config: langchain_core.runnables.RunnableConfig,
    ) -> FlightSearchState:
        """Handle flight search conversation using ReActAgent."""

        if not state["messages"]:
            initial_msg = langchain_core.messages.HumanMessage(content=state["query"])
            state["messages"].append(initial_msg)
            logger.info(f"Flight Query: {state['query']}")

        # Initialize parameter mapper
        parameter_mapper = ParameterMapper(self.chat_model)

        tools = []
        for tool_name in [
            "lookup_flight_info",
            "save_flight_booking",
            "retrieve_flight_bookings",
            "search_flight_policies",
        ]:
            catalog_tool = self.catalog.find("tool", name=tool_name)
            logger.info(f"Loaded tool: {tool_name}")

            def create_tool_wrapper(original_tool, name):
                def wrapper_func(tool_input: str) -> str:
                    """Wrapper to handle parameter mapping using ParameterMapper."""
                    try:
                        logger.debug(f"Tool wrapper called for {name} with input: '{tool_input}'")

                        # Use ParameterMapper to intelligently map string input to parameters
                        mapped_params = parameter_mapper.map_string_input(
                            name, tool_input, original_tool.func
                        )

                        logger.debug(f"Mapped parameters for {name}: {mapped_params}")

                        # Call the original tool with mapped parameters
                        result = original_tool.func(**mapped_params)

                        logger.debug(
                            f"Tool {name} result type: {type(result)}, length: {len(result) if hasattr(result, '__len__') else 'N/A'}"
                        )

                        return result

                    except openai.OpenAIError as e:
                        logger.warning(f"OpenAI service error in {name}: {e}")
                        return f"The {name.replace('_', ' ')} service is temporarily unavailable. Please try again or contact customer service."
                    except Exception as e:
                        logger.exception(f"Error in tool wrapper for {name}: {e!s}")
                        return f"Error calling {name}: {e!s}"

                return wrapper_func

            langchain_tool = Tool(
                name=tool_name,
                description=f"Tool for {tool_name.replace('_', ' ')}",
                func=create_tool_wrapper(catalog_tool, tool_name),
            )
            tools.append(langchain_tool)

        # Get prompt from Agent Catalog
        prompt_resource = self.catalog.find("prompt", name="flight_search_assistant")
        react_prompt = PromptTemplate.from_template(prompt_resource.content)

        # Create ReAct agent with tools and prompt
        agent = create_react_agent(self.chat_model, tools, react_prompt)

        # Create agent executor with optimized settings for Llama
        agent_executor = AgentExecutor(
            agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=8
        )

        # Execute the agent with enhanced error handling for Llama
        try:
            response = agent_executor.invoke({"input": state["query"]})
            output = response["output"]
        except openai.OpenAIError as e:
            # Handle OpenAI service errors (model unavailable, health errors, etc.)
            logger.warning(f"OpenAI service error in agent execution: {e}")
            output = "The flight search service is temporarily unavailable due to model maintenance. Please try again in a few minutes or contact customer service."
        except Exception as e:
            # Handle guardrail violations and other API errors gracefully
            error_msg = str(e)
            error_lower = error_msg.lower()

            # Check for guardrail violations with expanded patterns
            if (
                "guardrail_violation_error" in error_lower
                or "guardrail violation" in error_lower
                or "content policy" in error_lower
            ):
                # Treat guardrails as warnings, not errors
                logger.warning(f"Guardrails content moderated: {error_msg}")
                output = "I apologize, but I can't process that specific request due to content policies. Please try rephrasing your flight search query or ask about general flight information."

            # Handle timeout errors
            elif "timeout" in error_lower or "timed out" in error_lower:
                logger.warning(f"Request timeout: {error_msg}")
                output = "The request timed out. Please try again with a simpler query."

            # Handle connection errors
            elif "connection" in error_lower or "network" in error_lower:
                logger.warning(f"Connection error: {error_msg}")
                output = "I'm having trouble connecting to the flight database. Please try again in a moment."

            # Handle JSON/parsing errors
            elif "json" in error_lower or "parsing" in error_lower:
                logger.warning(f"Parsing error: {error_msg}")
                output = "I had trouble understanding the flight data. Please try rephrasing your request."

            # Handle API rate limiting
            elif "rate limit" in error_lower or "429" in error_msg:
                logger.warning(f"Rate limit exceeded: {error_msg}")
                output = "Too many requests. Please wait a moment before trying again."

            # Generic error fallback - don't break the app
            else:
                logger.warning(f"Unexpected error: {error_msg}")
                output = "I encountered an unexpected issue. Please try again or contact support if the problem persists."

            # Log the specific error type for debugging (but don't break the flow)
            if "guardrail" in error_lower:
                logger.info("Guardrails triggered - request handled gracefully")
            else:
                logger.info("Non-guardrail error - handled gracefully")

        # Add response to conversation
        assistant_msg = langchain_core.messages.AIMessage(content=output)
        state["messages"].append(assistant_msg)
        state["resolved"] = True

        return state


class FlightSearchGraph(agentc_langgraph.graph.GraphRunnable):
    """Flight search conversation graph."""

    @staticmethod
    def build_starting_state(query: str) -> FlightSearchState:
        """Build the initial state for the flight search."""
        return FlightSearchState(
            messages=[],
            query=query,
            resolved=False,
            search_results=[],
            previous_node=None,
        )

    def compile(self) -> langgraph.graph.graph.CompiledGraph:
        """Compile the LangGraph workflow."""
        search_agent = FlightSearchAgent(catalog=self.catalog, span=self.span)

        def flight_search_node(state: FlightSearchState) -> FlightSearchState:
            """Wrapper function for the flight search ReActAgent."""
            with self.span.new("Flight Search Node") as node_span:
                return search_agent._invoke(
                    span=node_span,
                    state=state,
                    config={},
                )

        workflow = langgraph.graph.StateGraph(FlightSearchState)
        workflow.add_node("flight_search", flight_search_node)
        workflow.set_entry_point("flight_search")
        workflow.add_edge("flight_search", langgraph.graph.END)

        return workflow.compile()



Agent classes defined


## Clear Flight Bookings

Clear existing bookings for clean test run.


In [7]:
def clear_flight_bookings():
    """Clear existing flight bookings to start fresh for demo."""
    try:
        client = CouchbaseClient(
            conn_string=os.getenv("CB_CONN_STRING", "couchbase://localhost"),
            username=os.getenv("CB_USERNAME", "Administrator"),
            password=os.getenv("CB_PASSWORD", "password"),
            bucket_name=os.getenv("CB_BUCKET", "travel-sample"),
        )
        client.connect()

        scope_name = "agentc_bookings"
        client.clear_scope(scope_name)
        logger.info("Cleared existing flight bookings for fresh test run")

    except Exception as e:
        logger.warning(f"Could not clear bookings: {e}")


# Clear existing bookings
clear_flight_bookings()

2025-07-13 12:42:25,218 - __main__ - INFO - Successfully connected to Couchbase
2025-07-13 12:42:28,389 - __main__ - INFO - Cleared collection 'user_bookings_20250713' in scope 'agentc_bookings'
2025-07-13 12:42:28,390 - __main__ - INFO - Cleared all collections in scope 'agentc_bookings'
2025-07-13 12:42:28,390 - __main__ - INFO - Cleared existing flight bookings for fresh test run


Cleared existing bookings


## Setup Flight Search Agent

Initialize the complete flight search agent setup.


In [8]:
def setup_flight_search_agent():
    """Setup flight search agent with all latest fixes."""
    try:
        setup_environment()

        # Initialize Agent Catalog
        catalog = agentc.Catalog(
            conn_string=os.environ["AGENT_CATALOG_CONN_STRING"],
            username=os.environ["AGENT_CATALOG_USERNAME"],
            password=os.environ["AGENT_CATALOG_PASSWORD"],
            bucket=os.environ["AGENT_CATALOG_BUCKET"],
        )
        application_span = catalog.Span(name="Flight Search Agent")

        with application_span.new("Couchbase Setup"):
            client = CouchbaseClient(
                conn_string=os.environ["CB_CONN_STRING"],
                username=os.environ["CB_USERNAME"],
                password=os.environ["CB_PASSWORD"],
                bucket_name=os.environ["CB_BUCKET"],
            )

            client.setup_collection(
                scope_name=os.environ["CB_SCOPE"], collection_name=os.environ["CB_COLLECTION"]
            )

        with application_span.new("Vector Store Setup"):
            embeddings = OpenAIEmbeddings(
                api_key=os.environ["OPENAI_API_KEY"], model="text-embedding-3-small"
            )
            try:
                CouchbaseVectorStore(
                    cluster=client.cluster,
                    bucket_name=os.environ["CB_BUCKET"],
                    scope_name=os.environ["CB_SCOPE"],
                    collection_name=os.environ["CB_COLLECTION"],
                    embedding=embeddings,
                    index_name=os.environ["CB_INDEX"],
                )
                logger.info("Vector store setup completed")
            except Exception as e:
                logger.warning(f"Vector store setup failed: {e}")

        with application_span.new("Agent Graph Creation"):
            flight_graph = FlightSearchGraph(catalog=catalog, span=application_span)
            compiled_graph = flight_graph.compile()

        logger.info("Agent Catalog integration successful")
        return compiled_graph, application_span

    except Exception as e:
        logger.exception(f"Setup error: {e}")
        logger.info("Ensure Agent Catalog is published: agentc index . && agentc publish")
        raise


# Setup the agent
compiled_graph, application_span = setup_flight_search_agent()

2025-07-13 12:42:28,400 - __main__ - INFO - Testing Capella AI connectivity...
2025-07-13 12:42:30,980 - __main__ - INFO - ✅ Capella AI embedding test successful - dimensions: 4096
2025-07-13 12:42:35,349 - __main__ - INFO - ✅ Capella AI LLM test successful
2025-07-13 12:42:35,350 - __main__ - INFO - ✅ Capella AI connectivity tests completed successfully
2025-07-13 12:42:37,472 - __main__ - INFO - Successfully connected to Couchbase
2025-07-13 12:42:38,574 - __main__ - INFO - Bucket 'travel-sample' exists
2025-07-13 12:42:44,475 - __main__ - INFO - Primary index created successfully
2025-07-13 12:42:44,475 - __main__ - INFO - Collection setup complete for shared.agentcatalog
2025-07-13 12:42:48,050 - __main__ - INFO - Vector store setup completed
2025-07-13 12:42:48,078 - __main__ - INFO - ✅ Using Capella AI for LLM
2025-07-13 12:43:08,425 - __main__ - INFO - Agent Catalog integration successful


Flight search agent setup complete


## Test Function

Define test function with better error handling.


In [9]:
def run_test_query(test_number: int, query: str):
    """Run a single test query with error handling."""
    with application_span.new(f"Test {test_number}: {query}") as query_span:
        logger.info(f"\n🔍 Test {test_number}: {query}")
        try:
            query_span["query"] = query
            state = FlightSearchGraph.build_starting_state(query=query)
            result = compiled_graph.invoke(state)
            query_span["result"] = result

            if result.get("search_results"):
                logger.info(f"Found {len(result['search_results'])} flight options")
            logger.info(f"Test {test_number} completed: {result.get('resolved', False)}")

            return result

        except Exception as e:
            logger.exception(f"❌ Test {test_number} failed: {e}")
            query_span["error"] = str(e)
            return None



Test function ready


## Test 1: Flight Search

Find flights from JFK to LAX for tomorrow - now with parameter mapping!


In [10]:
result1 = run_test_query(1, "Find flights from JFK to LAX for tomorrow")

2025-07-13 12:43:08,445 - __main__ - INFO - 
🔍 Test 1: Find flights from JFK to LAX for tomorrow
2025-07-13 12:43:08,455 - __main__ - INFO - Flight Query: Find flights from JFK to LAX for tomorrow
2025-07-13 12:43:08,474 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:43:08,485 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:43:08,496 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:43:08,507 - __main__ - INFO - Loaded tool: search_flight_policies




> Entering new AgentExecutor chain...
Thought: I need to search for flights between JFK and LAX for tomorrow.
Action: lookup_flight_info
Action Input: "JFK,LAX,tomorrow"
Observation['AS flight from JFK to LAX using 321 762', 'B6 flight from JFK to LAX using 320', 'DL flight from JFK to LAX using 76W 752', 'QF flight from JFK to LAX using 744', 'AA flight from JFK to LAX using 32B 762', 'UA flight from JFK to LAX using 757', 'US flight from JFK to LAX using 32B 762', 'VX flight from JFK to LAX using 320']Action: lookup_flight_info
Action Input: "JFK,LAX,tomorrow"
Observation['AS flight from JFK to LAX using 321 762', 'B6 flight from JFK to LAX using 320', 'DL flight from JFK to LAX using 76W 752', 'QF flight from JFK to LAX using 744', 'AA flight from JFK to LAX using 32B 762', 'UA flight from JFK to LAX using 757', 'US flight from JFK to LAX using 32B 762', 'VX flight from JFK to LAX using 320']Action: lookup_flight_info
Action Input: "JFK,LAX,tomorrow"
Observation['AS flight from JF

2025-07-13 12:43:17,006 - __main__ - INFO - Test 1 completed: True


Thought: I've tried searching for flights multiple times, but it seems like the tool is not providing any new information.
Action: 
Final Answer: There are multiple flights available from JFK to LAX for tomorrow, including AS, B6, DL, QF, AA, UA, US, and VX flights. However, I was unable to retrieve more detailed information about these flights. You can try checking the airlines' websites for more information or consider contacting the airlines directly for assistance.

> Finished chain.


## Test 2: Flight Booking (Business Class)

Book a flight with parameter mapping - no more ReAct artifacts!


In [11]:
result2 = run_test_query(
    2, "Book a flight from LAX to JFK for tomorrow, 2 passengers, business class"
)

2025-07-13 12:43:17,014 - __main__ - INFO - 
🔍 Test 2: Book a flight from LAX to JFK for tomorrow, 2 passengers, business class
2025-07-13 12:43:17,026 - __main__ - INFO - Flight Query: Book a flight from LAX to JFK for tomorrow, 2 passengers, business class
2025-07-13 12:43:17,041 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:43:17,052 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:43:17,064 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:43:17,075 - __main__ - INFO - Loaded tool: search_flight_policies




> Entering new AgentExecutor chain...
Thought: I need to find available flights from LAX to JFK for tomorrow.
Action: lookup_flight_info
Action Input: "LAX,JFK,tomorrow"
Observation['AS flight from LAX to JFK using 321 762', 'B6 flight from LAX to JFK using 320', 'DL flight from LAX to JFK using 76W 752', 'QF flight from LAX to JFK using 744', 'UA flight from LAX to JFK using 757', 'AA flight from LAX to JFK using 32B 762', 'US flight from LAX to JFK using 32B 762', 'VX flight from LAX to JFK using 320']Thought: Now that I have available flights, I need to create a booking for 2 passengers in business class.
Action: save_flight_booking
Action Input: "LAX,JFK,tomorrow,2,business"
ObservationFlight Booking Confirmed!

Booking ID: FL07146362AE62
Route: LAX → JFK
Departure Date: 2025-07-14
Return Date: One-way
Passengers: 2
Class: Business
Total Price: $1500.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid 

2025-07-13 12:43:27,539 - __main__ - INFO - Test 2 completed: True


Final Answer: Flight booked successfully. Booking ID: FL07146362AE62.

> Finished chain.


## Test 3: Flight Booking (Economy Class)

Book an economy flight - cleaning handles 'economy' correctly!


In [12]:
result3 = run_test_query(3, "Book an economy flight from JFK to MIA for next week, 1 passenger")

2025-07-13 12:43:27,552 - __main__ - INFO - 
🔍 Test 3: Book an economy flight from JFK to MIA for next week, 1 passenger
2025-07-13 12:43:27,554 - __main__ - INFO - Flight Query: Book an economy flight from JFK to MIA for next week, 1 passenger
2025-07-13 12:43:27,567 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:43:27,579 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:43:27,594 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:43:27,605 - __main__ - INFO - Loaded tool: search_flight_policies




> Entering new AgentExecutor chain...
Thought: I need to find available flights from JFK to MIA for next week.
Action: lookup_flight_info
Action Input: "JFK,MIA,next week"
Observation['DL flight from JFK to MIA using 73H M88 738 319', 'AA flight from JFK to MIA using 757 763 738', 'US flight from JFK to MIA using 757 738']Thought: I have found available flights from JFK to MIA for next week.
Action: save_flight_booking
Action Input: "JFK,MIA,next week,1,economy"
ObservationFlight Booking Confirmed!

Booking ID: FL0720024E2D93
Route: JFK → MIA
Departure Date: 2025-07-20
Return Date: One-way
Passengers: 1
Class: Economy
Total Price: $250.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid government-issued photo ID
4. Booking confirmation sent to your email

Thank you for choosing our airline!

2025-07-13 12:43:37,078 - __main__ - WARNING - No valid parameters mapped for retrieve_flight_bookings with input 'None
Observation'


Action: retrieve_flight_bookings
Action Input: None
ObservationYour Current Bookings (2 found):

Booking 1:
  Booking ID: FL0720024E2D93
  Route: JFK → MIA
  Date: 2025-07-20
  Passengers: 1
  Class: Economy
  Total: $250.00
  Status: confirmed
  Booked: 2025-07-13

Booking 2:
  Booking ID: FL07146362AE62
  Route: LAX → JFK
  Date: 2025-07-14
  Passengers: 2
  Class: Business
  Total: $1500.00
  Status: confirmed
  Booked: 2025-07-13

2025-07-13 12:43:50,195 - __main__ - INFO - Test 3 completed: True


Thought: I need to acknowledge the existing booking without repeating the action.
Final Answer: Your flight from JFK to MIA has been successfully booked for next week, and you can find the details in your current bookings.

> Finished chain.


## Test 4: Retrieve Current Bookings

Show current flight bookings with parameter handling.


In [13]:
result4 = run_test_query(4, "Show me my current flight bookings")

2025-07-13 12:43:50,202 - __main__ - INFO - 
🔍 Test 4: Show me my current flight bookings
2025-07-13 12:43:50,204 - __main__ - INFO - Flight Query: Show me my current flight bookings
2025-07-13 12:43:50,226 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:43:50,237 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:43:50,247 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:43:50,258 - __main__ - INFO - Loaded tool: search_flight_policies




> Entering new AgentExecutor chain...


2025-07-13 12:43:51,322 - __main__ - WARNING - No valid parameters mapped for retrieve_flight_bookings with input 'None
Observation'


Thought: I need to retrieve the existing flight bookings to show them to the user.
Action: retrieve_flight_bookings
Action Input: None
ObservationYour Current Bookings (2 found):

Booking 1:
  Booking ID: FL0720024E2D93
  Route: JFK → MIA
  Date: 2025-07-20
  Passengers: 1
  Class: Economy
  Total: $250.00
  Status: confirmed
  Booked: 2025-07-13

Booking 2:
  Booking ID: FL07146362AE62
  Route: LAX → JFK
  Date: 2025-07-14
  Passengers: 2
  Class: Business
  Total: $1500.00
  Status: confirmed
  Booked: 2025-07-13

2025-07-13 12:44:05,180 - __main__ - INFO - Test 4 completed: True


Thought: The user has existing bookings, so I don't need to perform any further actions.
Final Answer: Your Current Bookings (2 found):

Booking 1:
  Booking ID: FL0720024E2D93
  Route: JFK → MIA
  Date: 2025-07-20
  Passengers: 1
  Class: Economy
  Total: $250.00
  Status: confirmed
  Booked: 2025-07-13

Booking 2:
  Booking ID: FL07146362AE62
  Route: LAX → JFK
  Date: 2025-07-14
  Passengers: 2
  Class: Business
  Total: $1500.00
  Status: confirmed
  Booked: 2025-07-13

> Finished chain.


## Test 5: Flight Policy Search

Search flight policies with robust query handling.


In [14]:
result5 = run_test_query(5, "What are the baggage policies?")

2025-07-13 12:44:05,194 - __main__ - INFO - 
🔍 Test 5: What are the baggage policies?
2025-07-13 12:44:05,197 - __main__ - INFO - Flight Query: What are the baggage policies?
2025-07-13 12:44:05,212 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:44:05,224 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:44:05,273 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:44:05,331 - __main__ - INFO - Loaded tool: search_flight_policies




> Entering new AgentExecutor chain...
Thought: The user is asking about baggage policies, which could be related to checked baggage fees, carry-on restrictions, or other airline-specific rules.
Action: search_flight_policies
Action Input: "baggage policy"
ObservationPolicy 1:
Business Center Hotel in Downtown Chicago, Illinois. Modern business hotel with state-of-the-art conference facilities Price range: $180-$280 per night. Rating: 4.3/5. Amenities: 24/7 Business Center, Executive Meeting Rooms, Fitness Center, Business Lounge Restaurant, Free WiFi, Self-parking

Policy 2:
Business Center Hotel in Downtown Chicago, Illinois. Modern business hotel with state-of-the-art conference facilities Price range: $180-$280 per night. Rating: 4.3/5. Amenities: 24/7 Business Center, Executive Meeting Rooms, Fitness Center, Business Lounge Restaurant, Free WiFi, Self-parking

Policy 3:
Business Center Hotel in Downtown Chicago, Illinois. Modern business hotel with state-of-the-art conference fac

2025-07-13 12:45:22,203 - __main__ - INFO - Test 5 completed: True


Policy 1:
Grand Palace Hotel in Manhattan, New York City. Luxury 5-star hotel featuring elegant rooms with Manhattan skyline views Price range: $300-$500 per night. Rating: 4.8/5. Amenities: Rooftop Pool, World-class Spa, 24/7 Fitness Center, Michelin-starred Restaurant, 24/7 Room Service, High-speed WiFi

Policy 2:
Grand Palace Hotel in Manhattan, New York City. Luxury 5-star hotel featuring elegant rooms with Manhattan skyline views Price range: $300-$500 per night. Rating: 4.8/5. Amenities: Rooftop Pool, World-class Spa, 24/7 Fitness Center, Michelin-starred Restaurant, 24/7 Room Service, High-speed WiFi

Policy 3:
Grand Palace Hotel in Manhattan, New York City. Luxury 5-star hotel featuring elegant rooms with Manhattan skyline views Price range: $300-$500 per night. Rating: 4.8/5. Amenities: Rooftop Pool, World-class Spa, 24/7 Fitness Center, Michelin-starred Restaurant, 24/7 Room Service, High-speed WiFi

> Finished chain.


## Arize Phoenix Evaluation Demo

This section demonstrates how to evaluate the flight search agent using Arize Phoenix observability platform. The evaluation includes:

- **Relevance Scoring**: Using Phoenix RelevanceEvaluator to score how relevant responses are to queries
- **QA Scoring**: Using Phoenix QAEvaluator to score answer quality
- **Hallucination Detection**: Using Phoenix HallucinationEvaluator to detect fabricated information  
- **Toxicity Detection**: Using Phoenix ToxicityEvaluator to detect harmful content
- **Phoenix UI**: Real-time observability dashboard at `http://localhost:6006/`

We'll run two simple flight search queries and evaluate the responses for quality and safety. Note: We're only testing search functionality, not booking operations.

In [15]:
# Demo Arize Phoenix Evaluation for Flight Search Agent
# Run two simple flight search queries and evaluate responses

import builtins
import contextlib

import pandas as pd
import phoenix as px
from phoenix.evals import (
    HALLUCINATION_PROMPT_RAILS_MAP,
    HALLUCINATION_PROMPT_TEMPLATE,
    QA_PROMPT_RAILS_MAP,
    QA_PROMPT_TEMPLATE,
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    TOXICITY_PROMPT_RAILS_MAP,
    TOXICITY_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

# Start Phoenix session for observability
with contextlib.suppress(builtins.BaseException):
    px.launch_app(port=6006)

# Demo queries - simple search only (no bookings or complex operations)
flight_demo_queries = [
    "Find flights from JFK to LAX",
    "What flights are available from Miami to New York?"
]


/Users/kaustavghosh/Desktop/agent-catalog-quickstart/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-13 12:45:23,326 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/kaustavghosh/.phoenix
2025-07-13 12:45:23,341 - phoenix.inferences.inferences - INFO - Dataset: phoenix_inferences_da521529-576a-4243-b816-cdcbfc3a35ab initialized
2025-07-13 12:45:27,031 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: /Users/kaustavghosh/.phoenix
2025-07-13 12:45:27,093 - alembic.runtime.migration - INFO - Context impl SQLiteImpl.
2025-07-13 12:45:27,093 - alembic.runtime.migration - INFO - Will assume transactional DDL.
2025-07-13 12:45:27,110 - alembic.runtime.migration - INFO - Running upgrade  -> cf03bd6bae1d, init
2025-07-13 12:45:27,141 - alembic.runtime.migration -

❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.


2025-07-13 12:45:27,239 - alembic.runtime.migration - INFO - Running upgrade 2f9d1a65945f -> bb8139330879, create project trace retention policies table
2025-07-13 12:45:27,244 - alembic.runtime.migration - INFO - Running upgrade bb8139330879 -> 8a3764fe7f1a, change jsonb to json for prompts
2025-07-13 12:45:27,255 - alembic.runtime.migration - INFO - Running upgrade 8a3764fe7f1a -> 6a88424799fe, Add auth_method column to users table and migrate existing authentication data.
2025-07-13 12:45:27,262 - alembic.runtime.migration - INFO - Running upgrade 6a88424799fe -> a20694b15f82, Cost-related tables
2025-07-13 12:45:27,269 - phoenix.server.app - INFO - Server umap params: UMAPParameters(min_dist=0.0, n_neighbors=30, n_samples=500)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
✅ Phoenix UI started at http://localhost:6006
🔧 Setting up Phoenix evaluation demo for flight search...


llm_classify |          | 0/2 (0.0%) | ⏳ 00:03<? | ?it/s


In [16]:
# Run demo queries and collect responses for evaluation
flight_demo_results = []

for i, query in enumerate(flight_demo_queries, 1):

    try:
        # Create initial state and run the compiled graph
        state = FlightSearchGraph.build_starting_state(query=query)
        result = compiled_graph.invoke(state)

        # Extract the response from the final message
        if result.get("messages") and len(result["messages"]) > 1:
            output = result["messages"][-1].content
        else:
            output = "No response generated"

        flight_demo_results.append({
            "query": query,
            "response": output,
            "query_type": f"flight_demo_{i}",
            "success": result.get("resolved", False)
        })


    except Exception as e:
        flight_demo_results.append({
            "query": query,
            "response": f"Error: {e!s}",
            "query_type": f"flight_demo_{i}",
            "success": False
        })


2025-07-13 12:45:27,643 - __main__ - INFO - Flight Query: Find flights from JFK to LAX
2025-07-13 12:45:27,668 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:45:27,679 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:45:27,689 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:45:27,699 - __main__ - INFO - Loaded tool: search_flight_policies



🔍 Running Flight Demo Query 1: Find flights from JFK to LAX


> Entering new AgentExecutor chain...
Thought: I need to search for flights between JFK and LAX airports.
Action: lookup_flight_info
Action Input: "JFK,LAX,tomorrow"
Observation['AS flight from JFK to LAX using 321 762', 'B6 flight from JFK to LAX using 320', 'DL flight from JFK to LAX using 76W 752', 'QF flight from JFK to LAX using 744', 'AA flight from JFK to LAX using 32B 762', 'UA flight from JFK to LAX using 757', 'US flight from JFK to LAX using 32B 762', 'VX flight from JFK to LAX using 320']Action: lookup_flight_info
Action Input: "JFK,LAX,tomorrow"
Observation['AS flight from JFK to LAX using 321 762', 'B6 flight from JFK to LAX using 320', 'DL flight from JFK to LAX using 76W 752', 'QF flight from JFK to LAX using 744', 'AA flight from JFK to LAX using 32B 762', 'UA flight from JFK to LAX using 757', 'US flight from JFK to LAX using 32B 762', 'VX flight from JFK to LAX using 320']Action: lookup_flight_info
Action

2025-07-13 12:45:35,120 - __main__ - INFO - Flight Query: What flights are available from Miami to New York?
2025-07-13 12:45:35,144 - __main__ - INFO - Loaded tool: lookup_flight_info
2025-07-13 12:45:35,156 - __main__ - INFO - Loaded tool: save_flight_booking
2025-07-13 12:45:35,167 - __main__ - INFO - Loaded tool: retrieve_flight_bookings
2025-07-13 12:45:35,177 - __main__ - INFO - Loaded tool: search_flight_policies


Thought: I've tried searching for flights multiple times, but it seems like the tool is not working correctly.
Action: 
Final Answer: Unfortunately, I'm unable to find flights from JFK to LAX. You may want to try searching again later or contacting the airline directly for assistance.

> Finished chain.
✅ Response: Unfortunately, I'm unable to find flights from JFK to LAX. You may want to try searching again later...

🔍 Running Flight Demo Query 2: What flights are available from Miami to New York?


> Entering new AgentExecutor chain...
Thought: I need to find available flights from Miami to New York. 
Action: lookup_flight_info 
Action Input: "MIA, JFK, tomorrow"['DL flight from MIA to JFK using M88 73H 738 319', 'US flight from MIA to JFK using 763 757 738', 'AA flight from MIA to JFK using 763 757 738']Action: lookup_flight_info 
Action Input: "MIA, JFK, tomorrow"
Observation['DL flight from MIA to JFK using M88 73H 738 319', 'US flight from MIA to JFK using 763 757 738', 'AA fligh

In [17]:
# Run Phoenix evaluations on the flight search responses

# Convert to DataFrame for evaluation
flight_results_df = pd.DataFrame(flight_demo_results)

# Setup evaluator LLM (using OpenAI for consistency)
evaluator_llm = OpenAIModel(model="gpt-4o", temperature=0.1)

# Prepare evaluation data with proper column names for Phoenix evaluators
flight_eval_data = []
for _, row in flight_results_df.iterrows():
    flight_eval_data.append({
        "input": row["query"],
        "output": row["response"],
        "reference": "A helpful response about flights with specific flight information",
        "text": row["response"]  # For toxicity evaluation
    })

flight_eval_df = pd.DataFrame(flight_eval_data)

# Run individual evaluations with proper error handling
flight_relevance_results = llm_classify(
    dataframe=flight_eval_df[["input", "reference"]],
    model=evaluator_llm,
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())
)

flight_qa_results = llm_classify(
    dataframe=flight_eval_df[["input", "output", "reference"]],
    model=evaluator_llm,
    template=QA_PROMPT_TEMPLATE,
    rails=list(QA_PROMPT_RAILS_MAP.values())
)

flight_hallucination_results = llm_classify(
    dataframe=flight_eval_df[["input", "reference", "output"]],
    model=evaluator_llm,
    template=HALLUCINATION_PROMPT_TEMPLATE,
    rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values())
)

flight_toxicity_results = llm_classify(
    dataframe=flight_eval_df[["text"]],
    model=evaluator_llm,
    template=TOXICITY_PROMPT_TEMPLATE,
    rails=list(TOXICITY_PROMPT_RAILS_MAP.values())
)



/var/folders/1l/n1kj9bdj0ds6117tflwzmk8m0000gq/T/ipykernel_30989/2495970587.py:24: DeprecationWarning: `dataframe` argument is deprecated; use `data` instead
  flight_relevance_results = llm_classify(
2025-07-13 12:45:54,909 - phoenix.evals.executors - WARNING - 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
I0000 00:00:1752390954.911189 7877447 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


🧠 Running Phoenix AI evaluations on flight search responses...

📋 Running relevance evaluation...


llm_classify |██████████| 2/2 (100.0%) | ⏳ 00:04<00:00 |  2.37s/it
/var/folders/1l/n1kj9bdj0ds6117tflwzmk8m0000gq/T/ipykernel_30989/2495970587.py:32: DeprecationWarning: `dataframe` argument is deprecated; use `data` instead
  flight_qa_results = llm_classify(
2025-07-13 12:45:59,675 - phoenix.evals.executors - WARNING - 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


✅ Running QA evaluation...


llm_classify |██████████| 2/2 (100.0%) | ⏳ 00:07<00:00 |  3.94s/it
/var/folders/1l/n1kj9bdj0ds6117tflwzmk8m0000gq/T/ipykernel_30989/2495970587.py:40: DeprecationWarning: `dataframe` argument is deprecated; use `data` instead
  flight_hallucination_results = llm_classify(
2025-07-13 12:46:07,569 - phoenix.evals.executors - WARNING - 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🛡️ Running hallucination evaluation...


llm_classify |██████████| 2/2 (100.0%) | ⏳ 00:07<00:00 |  3.50s/it
/var/folders/1l/n1kj9bdj0ds6117tflwzmk8m0000gq/T/ipykernel_30989/2495970587.py:48: DeprecationWarning: `dataframe` argument is deprecated; use `data` instead
  flight_toxicity_results = llm_classify(
2025-07-13 12:46:14,593 - phoenix.evals.executors - WARNING - 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🔒 Running toxicity evaluation...


llm_classify |          | 0/2 (0.0%) | ⏳ 00:00<? | ?it/s 

Retries exhausted after 1 attempts: Missing template variable: 'input'

📊 Flight Search Agent Evaluation Results:
🎯 Relevance: {'relevant': 1, 'unrelated': 1}
✅ QA Quality: {'incorrect': 2}
🛡️ Hallucination: {'hallucinated': 2}
🔒 Toxicity: {}

🎉 Phoenix evaluation demo complete for flight search agent!
💡 Visit http://localhost:6006 to see detailed traces and evaluations
📊 The Phoenix UI shows LangGraph execution, tool calls, and evaluation scores
🔧 Compare hotel vs flight agent performance using the evaluation metrics above
